# Phase Estimation & Factoring — Detailed Notes (Session 7)
**Course:** CS490/5590 — Quantum Computing Applications in Data Science, AI, & Deep Learning  
**Instructor:** Luke Miller  

> **Purpose.**  These notes expand Session-7 slides into a self-contained study guide. You will learn how the Quantum Fourier Transform (QFT) extracts eigen-phases, how Quantum Phase Estimation (QPE) leverages repeated powers of a unitary, and how Shor’s algorithm wraps QPE to factor integers.  We also discuss noise-related challenges and AI/DS links. Mini-exercises with short answers appear at the end.

---

## Session Road-map
1. Quick recap (noise & transpilation)  
2. Quantum Fourier Transform (basics & circuit)  
3. Quantum Phase Estimation (QPE) algorithm & analysis  
4. Shor’s factoring algorithm (period finding)  
5. Modular arithmetic circuits  
6. Implementation pain-points on NISQ + mitigation strategies  
7. AI / DS applications & cryptographic implications  
8. Q&A  

---

## 0) Recap from Session 6  

| Topic | Take-away |
|-------|-----------|
| Decoherence, gate & read-out errors | Depth-heavy circuits like QPE require mitigation or fault-tolerance |
| Qiskit transpiler | Must reduce gate count to survive NISQ noise |
| Error-mitigation | ZNE, read-out calibration, dynamical decoupling |

*Connection to today*: QPE and Shor use **many** controlled-phase and modular-arithmetic gates; noise is the Achilles heel.

---

## 1) Quantum Fourier Transform (QFT)  

### 1.1 Definition  
For $n$ qubits ($N=2^{n}$)  
$$
\operatorname{QFT}\;|x\rangle
   \;=\;\frac1{\sqrt N}\sum_{y=0}^{N-1}e^{2\pi i x y/N}\,|y\rangle.
$$

### 1.2 Circuit pattern  

For qubit indices $q_{n-1}\ldots q_0$ (MSB…LSB):

```
q_{n-1}: ──H──R2──R3──…─RN───●────swap
q_{n-2}: ──────H──R2──…──────┼────swap
⋮
q_0    : ──────────H──────────●────
```
- `H` = Hadamard  
- $R_k$ = controlled phase $ \begin{pmatrix}1 & 0\\0 & e^{i2\pi/2^{k}}\end{pmatrix}$.

Depth $≈\!n(n-1)/2$ controlled phases → $O(n^2)$ gates.

**Inverse QFT** (IQFT) reverses gate order, replaces $R_k$ with $R_{-k}$, swaps back the qubit order.

---

## 2) Quantum Phase Estimation (QPE)

### 2.1 Problem statement  
Given unitary $U$ and *known* eigenstate $|\psi\rangle$ with $U|\psi\rangle=e^{2\pi i\theta}|\psi\rangle$, estimate $\theta$ to $n$ bits.

### 2.2 Registers  
- **Counting register**: $n$ qubits initialised to $|0\rangle^{\otimes n}$.  
- **Eigenstate register**: holds $|\psi\rangle$.

### 2.3 Algorithm  
1. **Hadamard layer**: $H^{\otimes n}$ → uniform superposition in counting register.  
2. **Controlled-powers**: for $k=0$ to $n-1$ (LSB first) apply controlled-$U^{2^{k}}$. This writes phase as relative phases.  
3. **Inverse QFT** on counting qubits.  
4. **Measure** counting register → integer $m ≈ 2^n \theta$.

**Precision**: with $n$ qubits, estimation error $\le 1/2^{n+1}$ with probability ≥ $4/\pi^2$ (can be boosted).

### 2.4 Resource counts  
- Controlled-$U^{2^{k}}$ may require $2^{k}$ repetitions of $U$ — exponential in precision unless $U^{2^{k}}$ can be implemented by arithmetic tricks.  
- QFT adds $O(n^2)$ 2-qubit gates.

---

## 3) Implementing QPE in Qiskit (toy example)

```python
from qiskit import QuantumCircuit, transpile
from math import pi

n = 3               # precision qubits
qc = QuantumCircuit(n+1, n)

# -- Prepare eigenstate: |1> is eigenstate of Z with phase π (θ=0.5)
qc.x(n)              # eigenstate qubit

# 1. Hadamards
qc.h(range(n))

# 2. Controlled Z^{2^k} == controlled-phase π·2^{k}
for k in range(n):
    angle = pi * (2 ** k)
    qc.cp(angle, k, n)

# 3. Inverse QFT
qc.append(QuantumCircuit(n).inverse().qft().to_gate(label='IQFT'), range(n))

# 4. Measure
qc.barrier()
qc.measure(range(n), range(n))

print(qc.draw('text'))
```
With ideal simulation you should read `100` (binary 4) representing θ = 0.5.

> **Noise note**: Each controlled-phase → two-qubit gate; fidelity falls quickly for $n>4$ on current hardware.

---

## 4) Shor’s factoring algorithm  

### 4.1 High-level roadmap  
1. Pick random $a<N$ coprime to $N$.  
2. Run **period-finding** on $f(x)=a^{x}\bmod N$ using QPE.  
   - Here $U$ is modular multiplication by $a$: $U|y\rangle=|a y\bmod N\rangle$.  
3. Extract period $r$ from measured phase via *continued fractions*.  
4. If $r$ even and $a^{r/2}\not\equiv -1\pmod N$, factors are  
   $\gcd(a^{r/2}\pm1,N)$.

### 4.2 Complexity  
- Modular exponentiation circuit: $O((\log N)^2)$ depth, $O((\log N)^3)$ gates.  
- Overall runtime $O((\log N)^3)$, polynomial in bits of $N$.

### 4.3 Modular exponentiation circuit pieces  
| Block | Gate count (Toffoli basis) |
|-------|---------------------------|
| Reversible add mod $N$ | $O((\log N)^2)$ |
| Controlled-mult mod $N$ | $O((\log N)^3)$ |
| Reusable ancillas | $O(\log N)$ |

Even for $N=21$ the circuit depth ≳ 1000 CNOTs → far beyond NISQ without heavy error-mitigation.

---

## 5) Noise & transpilation hurdles  

| Challenge | Mitigation / workaround |
|-----------|-------------------------|
| Deep IQFT | Use **approximate QFT**: drop small-angle rotations → reduces two-qubit gates 50-70 % at cost of modest precision. |
| Controlled $U^{2^{k}}$ loops | Use **repeat-unitary gadgets** to compress powers where algebraic properties permit (e.g., squaring mod $N$). |
| High qubit count | Simulate on Aer, run on hardware only for tiny $N$ (15, 21). |
| Coherent over-rotation | Calibrate $R_k$ gates, apply dynamical decoupling between large-angle phases. |

---

## 6) AI / Data-science relevance  

| Subroutine | Potential ML use |
|------------|------------------|
| QPE | Eigen-value estimation for covariance matrices (quantum PCA), Hamiltonian spectra in quantum chemistry for feature generation. |
| QFT-like blocks | Quantum signal processing for seasonal/time-series pattern detection. |
| Period-finding | Hidden-period discovery in data streams → anomaly detection. |
| Shor’s cryptanalysis | Forces development of *post-quantum* encryption for ML model privacy & federated learning. |

---

## 7) Worked algebra / proofs  

1. **QFT correctness**: show $F^\dagger F = I$ using geometric-series summation.  
2. **Phase kick-back**: controlled-$U$ on $|1\rangle$ control & eigenstate $|\psi\rangle$ produces phase $e^{2\pi i\theta}$ on control qubit.  
3. **Continued-fraction recovery**: if $ |\,\theta - s/r| < 1/(2r^2)$ then $s/r$ is the closest fraction with denominator ≤ $N$.  

---

## 8) Mini-exercises (solutions in Appendix)

1. Implement **approximate QFT** on 4 qubits by omitting $R_4$ and smaller; measure fidelity vs full QFT.  
2. Prove that controlled-$U^{2^{k}}$ can be built from $k$ copies of controlled-$U^{2^{k-1}}$ plus ancilla if $U$ commutes with itself (it does).  
3. For $N=15$, hand-execute Shor’s algorithm with $a=7$; show period $r=4$ and derive factors.  
4. Analyse effect of depolarising noise $p=0.01$ on 3-qubit QPE: compute success probability drop assuming each two-qubit gate fails independently.  
5. Show how eigen-phase estimation can find principal components of a 2-qubit density matrix (outline quantum PCA).

---

## 9) FAQ  

- **“Do I really need the exact eigenstate for QPE?”** You need a state with *non-zero overlap* on at least one eigenstate; success probability equals squared overlap.  
- **“Why is modular exponentiation so hard?”** Reversibility forbids discarding carries; you must uncompute ancillas, inflating gate count.  
- **“Why approximate QFT?”** Small-angle controlled rotations contribute little to phase precision but add many noisy two-qubit gates.  
- **“Can Shor break RSA today?”** No—device scale and error rates are far from what’s needed (thousands of logical qubits with error-correction).

---

## 10) Summary (Session 7)

- **QFT** maps computational basis phases → amplitude locations; $\Theta(n^2)$ gates.  
- **QPE** combines Hadamards, controlled powers of $U$, and inverse QFT to extract eigen-phases with exponential precision.  
- **Shor’s algorithm** uses QPE on modular-multiplication unitary to find periods and factor integers in polynomial time.  
- **Noise & depth** make practical execution challenging on NISQ; approximate QFT, transpilation, and mitigation help but full factoring needs fault-tolerance.  
- QPE technology underpins eigen-value-driven quantum ML and chemistry algorithms.

---

## 11) Looking ahead  

- **Next Session:** Quantum Approximate Optimisation Algorithm (QAOA) and combinatorial ML applications.  
- **Homework 3**:  
  - Code 3-qubit QPE for $U=Z$ (eigen-phase 0.5), test with depolarising noise.  
  - Hand-run Shor for $N=15$ (use mini-exercise 3).  
  - Compare full vs approximate QFT fidelity.

---

## Appendix — solutions to mini-exercises (sketch)

1. Fidelity $F \approx 0.97$ after dropping smallest rotations; depth shrinks by ~40 %.  
2. Induction on $k$: $U^{2^{k}} = (U^{2^{k-1}})^2$; double-controlled construction uses ancilla squared-flag.  
3. $7^2=49≡4$, $7^4=16≡1$ mod 15; $r=4$. Factors: $\gcd(7^{2}±1,15)=\gcd(49±1,15)=\gcd(48,15)=3,\; \gcd(50,15)=5$.  
4. Two-qubit gates $g=10$, success ≈ $(1-p)^g≈0.90$.  
5. Prepare density-matrix purification; apply QPE to controlled-SWAP Hamiltonian; measurement histogram gives eigenvalues.

